In [21]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.12.0


## Load data 

In [7]:
os.getcwd()

'/Users/oana/Documents/Stanford/Spring2023/CS230/final_project/cs230_project/code/tensorflow'

In [23]:
X_train = pd.read_csv('../../data/processed/X_train.csv')
y_train = pd.read_csv('../../data/processed/y_train.csv')

In [12]:
X_train.head()

,Rndrng_NPI,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Zip5,Tot_HCPCS_Cds,Tot_Benes,Tot_Srvcs,Tot_Sbmtd_Chrg,Tot_Mdcr_Alowd_Amt,Tot_Mdcr_Stdzd_Amt,Drug_Tot_HCPCS_Cds,...,Rndrng_Prvdr_Type_Slide Preparation Facility,Rndrng_Prvdr_Type_Speech Language Pathologist,Rndrng_Prvdr_Type_Sports Medicine,Rndrng_Prvdr_Type_Surgical Oncology,Rndrng_Prvdr_Type_Thoracic Surgery,Rndrng_Prvdr_Type_Undefined Physician type,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine,Rndrng_Prvdr_Type_Unknown Supplier/Provider Specialty,Rndrng_Prvdr_Type_Urology,Rndrng_Prvdr_Type_Vascular Surgery
0,1154385557,1,50266.0,51,637,4705.0,1438282.53,330963.03,271736.89,6.0,...,0,0,0,0,0,0,0,0,0,0
1,1336409887,0,10457.0,20,297,1256.0,147861.00,56649.57,40377.33,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1992947139,0,28480.0,6,54,3849.0,132698.92,109437.86,89860.75,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1710395058,0,53098.0,20,248,668.0,95485.00,34351.16,24858.06,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1346279965,1,31405.0,22,284,560.0,203149.00,62637.00,49052.22,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Remove provider ID from training data 
X_train = X_train.drop(['Rndrng_NPI'], axis=1)

In [38]:
train_features = np.array(X_train)

In [40]:
y_train['excluded']

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
1551182    0.0
1551183    0.0
1551184    0.0
1551185    0.0
1551186    0.0
Name: excluded, Length: 1551187, dtype: float64

In [41]:
train_labels = np.array(y_train['excluded'])

In [62]:
# Model starting point: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

In [63]:
EPOCHS = 100
BATCH_SIZE = 200

early_stopping = tf.keras.callbacks.EarlyStopping(
    #monitor='val_prc', 
    verbose=1,
    #patience=10,
    mode='max',
    restore_best_weights=True)

In [64]:
basic_model = make_model()
basic_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 16)                2336      
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 2,353
Trainable params: 2,353
Non-trainable params: 0
_________________________________________________________________


In [67]:
results = basic_model.evaluate(train_features, 
                               train_labels, 
                               batch_size=BATCH_SIZE, 
                               verbose=1)

7756/7756 [==============================] - 9s 1ms/step - loss: nan - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 1550762.0000 - fn: 425.0000 - accuracy: 0.9997 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 1.9112e-04 - prc: 3.2029e-06


In [66]:
results

[nan,
 0.0,
 0.0,
 1550762.0,
 425.0,
 0.9997259974479675,
 0.0,
 0.0,
 0.00019112425798084587,
 3.2028813166107284e-06]

In [55]:
print("Loss: {:0.4f}".format(results[0]))

Loss: nan


In [ ]:
# Set output layer's bias to reflect inbalanced data
# initial_bias = np.log([pos/neg])
